In [1]:
import os

In [2]:
%pwd

'e:\\My Projects\\NLP\\Projects\\TextSummarization\\TextSummarization\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'e:\\My Projects\\NLP\\Projects\\TextSummarization\\TextSummarization'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen = True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: Path
    local_data_file : Path
    unzip_dir: Path

    


In [6]:
from TextSummarizer.constants import *
from TextSummarizer.utils.common import read_yaml, create_directories

In [19]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [10]:
! pip install gdown 

  Using cached gdown-5.1.0-py3-none-any.whl.metadata (5.7 kB)
  Using cached PySocks-1.7.1-py3-none-any.whl (16 kB)
Using cached gdown-5.1.0-py3-none-any.whl (17 kB)


In [12]:
import os
import zipfile
import gdown
from TextSummarizer.logging import logger
from TextSummarizer.utils.common import get_size

In [20]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self)-> str:
        '''
        Fetch data from the url
        '''

        try: 
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion", exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")

            file_id = dataset_url.split("/")[-2]
            prefix = 'https://drive.google.com/uc?/export=download&id='
            gdown.download(prefix+file_id,zip_download_dir)

            logger.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")

        except Exception as e:
            raise e
        


    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [22]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-02-21 12:15:57,799: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-02-21 12:15:57,799: INFO: common: yaml file: params.yaml loaded successfully]
[2024-02-21 12:15:57,805: INFO: common: created directory at: artifacts]
[2024-02-21 12:15:57,808: INFO: common: created directory at: artifacts/data_ingestion]
[2024-02-21 12:15:57,809: INFO: 1803536212: Downloading data from https://drive.google.com/file/d/16KtdK59azvffGWaXwLbWrjwOx9kZDYUZ/view?usp=sharing into file artifacts/data_ingestion/data.zip]


Downloading...
From (original): https://drive.google.com/uc?/export=download&id=16KtdK59azvffGWaXwLbWrjwOx9kZDYUZ
From (redirected): https://drive.google.com/uc?%2Fexport=download&id=16KtdK59azvffGWaXwLbWrjwOx9kZDYUZ&confirm=t&uuid=4c96fae2-3816-4307-8a5a-4d2ca8b24252
To: e:\My Projects\NLP\Projects\TextSummarization\TextSummarization\artifacts\data_ingestion\data.zip
100%|██████████| 27.0M/27.0M [00:07<00:00, 3.82MB/s]

[2024-02-21 12:16:08,824: INFO: 1803536212: Downloaded data from https://drive.google.com/file/d/16KtdK59azvffGWaXwLbWrjwOx9kZDYUZ/view?usp=sharing into file artifacts/data_ingestion/data.zip]
